# Математика - Домашнее задание 7

**Дедлайн:** 23:59, 17 января

**Задание:**
Для датасета из прошлого домашнего задания необходимо:

1. Сформировать двухвыборочные гипотезы касательно медиан и распределений для дискретного и непрерывного случая. Если дискретного показателя нет - создайте его дополнительно. Итого 4 гипотезы (4 пункта)
2. Каждую гипотезу проверить релевантным тестом и обосновать, почему выбрали именно его. 5-м пунктом будет проверка результатов с бутстрапом
3. Сравнить результаты из 4 пунктов с результатами бутстрапа и обосновать различия, если они есть
4. Определить какой подход мощнее в конкретном случае и почему

**Для получения зачета необходимо:**
1. **Зачет на 4:** выполнены 4 пункта без сравнения с бутстрапом
2. **Зачет на 5:** выполнены все пункты с небольшими недочетами

#### Библиотеки

In [4]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.stats import ttest_ind, mannwhitneyu, norm, shapiro, normaltest, ttest_ind_from_stats, kstest, anderson, normaltest, chisquare, median_test
from itertools import combinations
import scipy.stats as sts
from scipy import stats

# Настройка стиля
sns.set_style("whitegrid")  
palette = sns.color_palette("husl", 8)
%matplotlib inline

## Подготовка данных

#### **Predict Online Course Engagement Dataset**
**Understanding User Behavior and Course Completion**



[Link](https://www.kaggle.com/datasets/rabieelkharoua/predict-online-course-engagement-dataset)

In [775]:
# Загружаю данные
df = pd.read_csv('online_course_engagement_data.csv')
df.head(10)

# Удалю переменные, на которые не хочу смотреть

df = df.drop(['CourseCompletion', 'DeviceType'], axis=1)

#### Новые метрики

In [777]:
# Average Time Spent Per Video
# TimeSpentOnCourse/NumberOfVideosWatched

df['AverageTimeSpentPerVideo'] = np.where(
    df['NumberOfVideosWatched'] > 0,
    df['TimeSpentOnCourse'] / df['NumberOfVideosWatched'],
    0
)
# Average Quiz Score
# QuizScores/NumberOfQuizzesTaken

df['AverageQuizScore'] = np.where(
    df['NumberOfQuizzesTaken'] > 0,
    df['QuizScores'] / df['NumberOfQuizzesTaken'],
    0
)

In [778]:
# Рассчитываем AverageTimeSpentPerVideo и AverageQuizScore

df = df[(df['CompletionRate'] != 0)]

# И удалю дубликаты по id и по курсу - точно один и тот же пользователь
df= df.drop_duplicates(subset=['UserID', 'CourseCategory'], keep=False)

# Удалю переменные, на которые не хочу смотреть

df = df.drop(['NumberOfVideosWatched', 'NumberOfQuizzesTaken', 'QuizScores', ], axis=1)
df

,UserID,CourseCategory,TimeSpentOnCourse,CompletionRate,AverageTimeSpentPerVideo,AverageQuizScore
0,5618,Health,29.979719,20.860773,1.763513,16.788552
1,4326,Arts,27.802640,65.632415,27.802640,12.523194
2,5849,Arts,86.820485,63.812007,6.201463,39.229481
3,4992,Science,35.038427,95.433162,2.061084,5.919885
4,3866,Programming,92.490647,18.102478,5.780665,0.000000
...,...,...,...,...,...,...
8995,8757,Health,37.445225,32.990704,2.674659,13.617340
8996,894,Science,48.631443,0.254625,6.947349,8.487608
8997,6323,Health,38.212512,70.188159,12.737504,23.169432
8998,3652,Health,70.048665,72.975225,5.388359,7.965518


#### Создам дискретный показатель. Пусть это будет показатель, характеризующий оценку пользователем курса, который он проходит, по шкале от 1 до 10.

In [780]:
np.random.seed(42)
df["UserRating"] = np.random.randint(1, 11, size=len(df))
df

,UserID,CourseCategory,TimeSpentOnCourse,CompletionRate,AverageTimeSpentPerVideo,AverageQuizScore,UserRating
0,5618,Health,29.979719,20.860773,1.763513,16.788552,7
1,4326,Arts,27.802640,65.632415,27.802640,12.523194,4
2,5849,Arts,86.820485,63.812007,6.201463,39.229481,8
3,4992,Science,35.038427,95.433162,2.061084,5.919885,5
4,3866,Programming,92.490647,18.102478,5.780665,0.000000,7
...,...,...,...,...,...,...,...
8995,8757,Health,37.445225,32.990704,2.674659,13.617340,5
8996,894,Science,48.631443,0.254625,6.947349,8.487608,10
8997,6323,Health,38.212512,70.188159,12.737504,23.169432,9
8998,3652,Health,70.048665,72.975225,5.388359,7.965518,1


Разделим аудиторию на сегменты по категории курса. Напомню: 'Health', 'Arts', 'Science', 'Programming', 'Business'.
**Данные итак агрегированные, нет смысла в группировке - id повторяются только для одинаковых id при разных CourseCategory, то есть, если человек проходил, например, сразу два курса.**

In [782]:
business_sample = df.query('CourseCategory == "Business"')

health_sample = df.query('CourseCategory == "Health"')

arts_sample = df.query('CourseCategory == "Arts"')

science_sample = df.query('CourseCategory == "Science"')

programming_sample = df.query('CourseCategory == "Programming"')

## 1. Сформировать двухвыборочные гипотезы касательно медиан и распределений для дискретного и непрерывного случая. Если дискретного показателя нет - создайте его дополнительно. Итого 4 гипотезы (4 пункта)

#### Выборки являются несвязанными 

##### (но лишь частично связанными - так как один и тот же пользователь может встречаться сразу в нескольких категориях курсов (можно аналитически в этом убедиться, посмотрев дубли пользователей и какой категории они соответствуют)).

#### Предварительно давайте убедимся в соответствии наших данных нормальному распределению, а также соответствию выборочных средних нормальному распределению.

In [786]:
def check_normality(df, column_name):
    values = df[column_name].dropna().values
    stat, p = shapiro(values)
    if p >= 0.05:
        print(f"{column_name}: Данные распределены нормально (p = {p:.5f})")
    else:
        print(f"{column_name}: Данные не распределены нормально (p = {p:.5f})")

quantitative_columns = [
    "TimeSpentOnCourse", "CompletionRate", "AverageTimeSpentPerVideo",
    "AverageQuizScore"
]

# Применяем проверку на нормальность для каждого столбца
for col in quantitative_columns:
    check_normality(df, col)

# Функция для проверки нормальности выборочных средних
def check_normality_of_means(df, group_column, value_column):
    group_means = df.groupby(group_column)[value_column].mean().values
    stat, p = shapiro(group_means)
    if p > 0.05:
        print(f"Средние значения {value_column} распределены нормально (p = {p:.5f})")
    else:
        print(f"Средние значения {value_column} не распределены нормально (p = {p:.5f})")

group_column = "CourseCategory"
quantitative_columns = [
    "TimeSpentOnCourse", "CompletionRate", "AverageTimeSpentPerVideo",
    "AverageQuizScore"
]
for col in quantitative_columns:
    check_normality_of_means(df, group_column, col)

TimeSpentOnCourse: Данные не распределены нормально (p = 0.00000)
CompletionRate: Данные не распределены нормально (p = 0.00000)
AverageTimeSpentPerVideo: Данные не распределены нормально (p = 0.00000)
AverageQuizScore: Данные не распределены нормально (p = 0.00000)
Средние значения TimeSpentOnCourse распределены нормально (p = 0.99262)
Средние значения CompletionRate распределены нормально (p = 0.80160)
Средние значения AverageTimeSpentPerVideo распределены нормально (p = 0.24710)
Средние значения AverageQuizScore распределены нормально (p = 0.19750)


D:\Anaconda\Lib\site-packages\scipy\stats\_axis_nan_policy.py:573: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 7246.
  res = hypotest_fun_out(*samples, **kwds)


### Формируем гипотезы!!!

Помним, что выборки являются несвязанными / данные не распределены нормально / средние не распределены нормально при выборе соответствующего критерия.
Предполагаю, что данные взяты из распределения с одной формой (смотрим на один и тот же показатель в рамках одной и той же платформы в разрезе крайне схожих продуктов)

$\alpha = 0.05$

Если $p < \alpha$, то нулевая гипотеза отвергается в пользу альтернативы. Если $p \ge \alpha$, то нулевая гипотеза не отвергается.

Давайте сравним моменты для категории здоровья и категории бизнеса. Health & Business.

#### Двухвыборочная гипотеза о медиане для непрерывного случая (**Критерий Уилкоксона-Манна-Уитни**):

Подходит, так как оценим значимость сдвига распределений + для независимых выборок.

Будем смотреть на time spent on course - проведенное за прохождением курса время.

Конкретизируем гипотезу как:

$H_0$ - $F_X(x) = F_Y(x)$ (разница в медианных значениях проведенного над курсом времени по выборкам клиентов по категории бизнес курсы и курсы по здоровью НЕ ЯВЛЯЕТСЯ статистически значимой (подробнее 0 между выборками X и Y нет статистически значимого сдвига, где выборка X - business_sample, выборка Y - health_sample))

$H_1$ - $F_X(x) = F_Y(x+m), m\neq0$ (разница в медианных значениях проведенного над курсом времени по выборкам клиентов по категории бизнес курсы и курсы по здоровью ЯВЛЯЕТСЯ статистически значимой; между выборками X и Y есть статистически значимый сдвиг)


#### Двухвыборочная гипотеза о распределении для непрерывного случая (**Двувыборочный тест Колмогорова-Смирнова**):

Посмотрим на схожесть распределений в двух выборках (для непрерывного случая)

Будем смотреть на time spent on course - проведенное за прохождением курса время.

ПОЧЕМУ? Критерий работает для непрерывных распределений

**ФОРМУЛИРОВКА:** Идентичны ли распределения в двух выборках (выборках business_sample и health_sample)?

$H_0$ - выборки проведенного над курсом времени по выборкам клиентов по категории бизнес курсы и курсы по здоровью к нам пришли из некоторого распределения $F_0$, идентичного по параметрам.

$H_1$ - выборки проведенного над курсом времени по выборкам клиентов по категории бизнес курсы и курсы по здоровью к нам НЕ пришли из некоторого распределения $F_0$, идентичного по параметрам.


#### Двухвыборочная гипотеза о распределении для дискретного случая (**Критерий Пирсона**):

**Гипотеза о неизвестном законе распределения:** 

$H_0$ - выборки оценок пользователей (UserRating) по выборкам клиентов по категории бизнес курсы и курсы по здоровью к нам пришли из некоторого распределения $F_0$, идентичного по параметрам.

$H_1$ - выборки оценок пользователей (UserRating) по выборкам клиентов по категории бизнес курсы и курсы по здоровью к нам НЕ пришли из некоторого распределения $F_0$, идентичного по параметрам.

$$
\begin{aligned}
H_0 : X \sim F_0 \\
H_1 : X \nsim F_0
\end{aligned}
$$

#### Двухвыборочная гипотеза о медиане для дискретного случая (**Тест Муда**):

Хочу взять Median Non Parametric Hypothesis Test

"This test works when the dependent variable is continuous or discrete count, and the independent variables are discrete with two or more attributes."

$H_0$ - выборки оценок пользователей (UserRating) по выборкам клиентов по категории бизнес курсы и курсы по здоровью имеют одинаковые медианные значения.

$H_1$ - выборки оценок пользователей (UserRating) по выборкам клиентов по категории бизнес курсы и курсы по здоровью имеют разные медианные значения.

$H_0: {Med}(X_1 - Y_1) = 0$,

$H_1: {Med}(X_1 - Y_1) \neq 0$

## 2. Каждую гипотезу проверить релевантным тестом и обосновать, почему выбрали именно его. 5-м пунктом будет проверка результатов с бутстрапом

### Объяснил, почему выбрал в пункте 1.
Сейчас проверю

#### Двухвыборочная гипотеза о медиане для непрерывного случая (**Критерий Уилкоксона-Манна-Уитни**):

$H_0$ - $F_X(x) = F_Y(x)$ (разница в медианных значениях проведенного над курсом времени по выборкам клиентов по категории бизнес курсы и курсы по здоровью не являтся статистически значимой (подробнее 0 между выборками X и Y нет статистически значимого сдвига, где выборка X - business_sample, выборка Y - health_sample))

$H_1$ - $F_X(x) = F_Y(x+m), m\neq0$ (разница в медианных значениях проведенного над курсом времени по выборкам клиентов по категории бизнес курсы и курсы по здоровью являтся статистически значимой; между выборками X и Y есть статистически значимый сдвиг)

In [791]:
stats.mannwhitneyu(business_sample['TimeSpentOnCourse'], health_sample['TimeSpentOnCourse'], alternative='two-sided')

# pvalue=0.7120347935080252 - выборки пришли из одного распределения и имеют схожие параметры среднего и технически медианы

MannwhitneyuResult(statistic=1060184.0, pvalue=0.7120347935080252)

#### Двухвыборочная гипотеза о распределении для непрерывного случая (**Двувыборочный тест Колмогорова-Смирнова**):

$H_0$ - выборки проведенного над курсом времени по выборкам клиентов по категории бизнес курсы и курсы по здоровью к нам пришли из некоторого распределения $F_0$, идентичного по параметрам.

$H_1$ - выборки проведенного над курсом времени по выборкам клиентов по категории бизнес курсы и курсы по здоровью к нам НЕ пришли из некоторого распределения $F_0$, идентичного по параметрам.


In [2]:
stats.ks_2samp(business_sample['TimeSpentOnCourse'], health_sample['TimeSpentOnCourse'], alternative='two-sided')

# pvalue=0.9203308054404328 - выборки пришли из одного распределения.

NameError: name 'sts' is not defined

#### Двухвыборочная гипотеза о распределении для дискретного случая (**Критерий Пирсона**):

$H_0$ - выборки оценок пользователей (UserRating) по выборкам клиентов по категории бизнес курсы и курсы по здоровью к нам пришли из некоторого распределения $F_0$, идентичного по параметрам.

$H_1$ - выборки оценок пользователей (UserRating) по выборкам клиентов по категории бизнес курсы и курсы по здоровью к нам НЕ пришли из некоторого распределения $F_0$, идентичного по параметрам.

$$
\begin{aligned}
H_0 : X \sim F_0 \\
H_1 : X \nsim F_0
\end{aligned}
$$

In [795]:
# chisquare(f_obs, f_exp=None, ddof=0, axis=0, *, sum_check=True)

# Частоты для каждой категории

business_ratings = business_sample['UserRating']
health_ratings = health_sample['UserRating']

business_counts = business_ratings.value_counts().sort_index()
health_counts = health_ratings.value_counts().sort_index()

# Формируем таблицу сопряженности
contingency_table = pd.DataFrame({
    'Health': health_counts,
    'Business': business_counts
})

contingency_table

,Health,Business
UserRating,,
1,144,143
2,126,152
3,140,155
4,147,123
5,155,152
6,136,150
7,144,162
8,147,141
9,151,159


In [796]:
stats.chi2_contingency(contingency_table.T)

# pvalue=0.5238797552462997 - не отвергаем нулевую гипотезу, выборки пришли из одного распределения

Chi2ContingencyResult(statistic=8.102209305024596, pvalue=0.5238797552462997, dof=9, expected_freq=array([[142.02770178, 137.57387141, 145.98666211, 133.61491108,
        151.9251026 , 141.53283174, 151.43023256, 142.52257182,
        153.40971272, 146.97640219],
       [144.97229822, 140.42612859, 149.01333789, 136.38508892,
        155.0748974 , 144.46716826, 154.56976744, 145.47742818,
        156.59028728, 150.02359781]]))

#### Двухвыборочная гипотеза о медиане для дискретного случая (**Тест Муда**):

$H_0$ - выборки оценок пользователей (UserRating) по выборкам клиентов по категории бизнес курсы и курсы по здоровью имеют одинаковые медианные значения.

$H_1$ - выборки оценок пользователей (UserRating) по выборкам клиентов по категории бизнес курсы и курсы по здоровью имеют разные медианные значения.

$H_0: {Med}(X_1 - Y_1) = 0$,

$H_1: {Med}(X_1 - Y_1) \neq 0$

In [798]:
business_ratings = business_sample['UserRating']

health_ratings = health_sample['UserRating']

# Применяем тест Муда

stats.median_test(health_ratings, business_ratings)

# pvalue=0.7543960616599104 - медианные значения одинаковые

MedianTestResult(statistic=0.09787394034687456, pvalue=0.7543960616599104, median=6.0, table=array([[599, 602],
       [848, 875]], dtype=int64))

### Сводная таблица

| Гипотеза                                  | Метод                                   | p-значение |
| ----------------------------------------- | --------------------------------------- | ---------- |
| **Медиана для непрерывного случая**       | Критерий Уилкоксона-Манна-Уитни         | 0.7120     |
| **Распределение для непрерывного случая** | Двувыборочный тест Колмогорова-Смирнова | 0.9203     |
| **Распределение для дискретного случая**  | Критерий Пирсона                        | 0.5239     |
| **Медиана для дискретного случая**        | Тест Муда                               | 0.7544     |

## 3. Сравнить результаты из 4 пунктов с результатами бутстрапа и обосновать различия, если они есть

### Бутстрап для оценки медианы по непрерывной колонке

In [802]:
def bootstrap_median(sample1, sample2):
    bootstrap_median_diff = [] 
    for _ in range(10000):
        sample1_resampled = np.random.choice(sample1, size=len(sample1), replace=True)
        sample2_resampled = np.random.choice(sample2, size=len(sample2), replace=True)
        median_diff = np.median(sample1_resampled) - np.median(sample2_resampled)
        bootstrap_median_diff.append(median_diff)
    return bootstrap_median_diff

In [803]:
np.random.seed(42)

boot_median1 = bootstrap_median(business_sample['TimeSpentOnCourse'], health_sample['TimeSpentOnCourse'])

# Доверительный интервал
big_ci = np.percentile(boot_median1, q=[2.5, 97.5]).round(2)
print(f"95% доверительный интервал для boot_median1: {big_ci}")

print(stats.mannwhitneyu(business_sample['TimeSpentOnCourse'], health_sample['TimeSpentOnCourse'], alternative='two-sided'))

95% доверительный интервал для boot_median1: [-3.51  3.39]
MannwhitneyuResult(statistic=1060184.0, pvalue=0.7120347935080252)


### Бутстрап для оценки распределения по непрерывной колонке

In [805]:
def bootstrap_mean(sample1, sample2):
    bootstrap_mean_diff = [] 
    for _ in range(10000):
        sample1_resampled = np.random.choice(sample1, size=len(sample1), replace=True)
        sample2_resampled = np.random.choice(sample2, size=len(sample2), replace=True)
        mean_diff = np.mean(sample1_resampled) - np.mean(sample2_resampled)
        bootstrap_mean_diff.append(mean_diff)
    return bootstrap_mean_diff

In [806]:
np.random.seed(42)

boot_mean1 = bootstrap_mean(business_sample['TimeSpentOnCourse'], health_sample['TimeSpentOnCourse'])

# Доверительный интервал
big_ci = np.percentile(boot_mean1, q=[2.5, 97.5]).round(2)
print(f"95% доверительный интервал для boot_mean1: {big_ci}")

print(stats.ks_2samp(business_sample['TimeSpentOnCourse'], health_sample['TimeSpentOnCourse'], alternative='two-sided'))

95% доверительный интервал для boot_mean1: [-2.39  1.66]
KstestResult(statistic=0.020117264538636423, pvalue=0.9203308054404328, statistic_location=21.318158751250607, statistic_sign=1)


### Бутстрап для оценки медианы по дискретной колонке

In [808]:
def bootstrap_median(sample1, sample2):
    bootstrap_median_diff = [] 
    for _ in range(10000):
        sample1_resampled = np.random.choice(sample1, size=len(sample1), replace=True)
        sample2_resampled = np.random.choice(sample2, size=len(sample2), replace=True)
        median_diff = np.median(sample1_resampled) - np.median(sample2_resampled)
        bootstrap_median_diff.append(median_diff)
    return bootstrap_median_diff

In [809]:
np.random.seed(42)

boot_median2 = bootstrap_median(business_sample['UserRating'], health_sample['UserRating'])

big_ci = np.percentile(boot_median2, q=[2.5, 97.5]).round(2)
print(f"95% доверительный интервал для boot_median2: {big_ci}")

print(stats.median_test(health_ratings, business_ratings))

95% доверительный интервал для boot_median2: [-1.  1.]
MedianTestResult(statistic=0.09787394034687456, pvalue=0.7543960616599104, median=6.0, table=array([[599, 602],
       [848, 875]], dtype=int64))


### Бутстрап для оценки распределения по дискретной колонке

In [811]:
def bootstrap_mean(sample1, sample2):
    bootstrap_mean_diff = [] 
    for _ in range(10000):
        sample1_resampled = np.random.choice(sample1, size=len(sample1), replace=True)
        sample2_resampled = np.random.choice(sample2, size=len(sample2), replace=True)
        mean_diff = np.mean(sample1_resampled) - np.mean(sample2_resampled)
        bootstrap_mean_diff.append(mean_diff)
    return bootstrap_mean_diff

In [812]:
np.random.seed(42)

boot_mean2 = bootstrap_mean(business_sample['UserRating'], health_sample['UserRating'])

big_ci = np.percentile(boot_mean2, q=[2.5, 97.5]).round(2)
print(f"95% доверительный интервал для boot_mean2: {big_ci}")

print(stats.chi2_contingency(contingency_table.T))

95% доверительный интервал для boot_mean2: [-0.28  0.13]
Chi2ContingencyResult(statistic=8.102209305024596, pvalue=0.5238797552462997, dof=9, expected_freq=array([[142.02770178, 137.57387141, 145.98666211, 133.61491108,
        151.9251026 , 141.53283174, 151.43023256, 142.52257182,
        153.40971272, 146.97640219],
       [144.97229822, 140.42612859, 149.01333789, 136.38508892,
        155.0748974 , 144.46716826, 154.56976744, 145.47742818,
        156.59028728, 150.02359781]]))


## Сводная таблица

| **Случай**       | **На что смотрим** | **95% доверительный бутстрап-интервал для разницы** | **Статистический тест**                              | **$H_0$ принимается** |
|-------------------|--------------------|-----------------------------------------------------|------------------------------------------------------|---------------------|
| Непрерывная       | Медиана            | [-3.51, 3.39]                                      | Mann-Whitney U (Критерий Уилкоксона-Манна-Уитни)     | Да                  |
| Непрерывная       | Распределение      | [-2.39, 1.66]                                      | Kolmogorov-Smirnov                                   | Да                  |
| Дискретная        | Медиана            | [-1, 1]                                            | Median (Mood's) Test                                 | Да                  |
| Дискретная        | Распределение      | [-0.28, 0.13]                                      | Chi-squared                                          | Да                  |


| Гипотеза                                  | Метод                                   | p-значение | **$H_0$ принимается** |
| ----------------------------------------- | --------------------------------------- | ---------- |---------------------|
| **Медиана для непрерывного случая**       | Критерий Уилкоксона-Манна-Уитни         | 0.7120     | Да                  |
| **Распределение для непрерывного случая** | Двувыборочный тест Колмогорова-Смирнова | 0.9203     | Да                  |
| **Распределение для дискретного случая**  | Критерий Пирсона                        | 0.5239     | Да                  |
| **Медиана для дискретного случая**        | Тест Муда                               | 0.7544     | Да                  |


#### Так или иначе, каждый доверительный интервал включает в себя 0, что говорит о том, что статистически значимых различий между группами нет, так как с заданной 95% доверительной вероятностью доверительный интервал охватывает истинное значение момента. Смотря на анализируемые параметры (для распределений генерировал средние в бутстрапе для более корректного подхода), можно делать релевантные выводы касательно того, что распределения выборок имеют определенное сходство, ровно как и заданный в гипотезах момент (медиана).

## 4. Определить какой подход мощнее в конкретном случае и почему

Выводы: в данном случае оба подхода дают согласующиеся результаты.
Тем не менее, релевантным предположением будет то, что доверительный интервал мощнее, если нам необходимо понять практическую суть-значимость разницы. Если цель заключается лишь в отвержении/не отвержении гипотезы, то рассматриваем p-value.
Хотя, в целом, оба подхода показали, что мелкие различия в выборках им достаточно трудно выявлять.